<a href="https://colab.research.google.com/github/telmacarvalho/pi-4-univesp/blob/main/treinamento_ml_pi_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinamento do Modelo Multi-Target com RandomForestRegressor

## Importação de bibliotecas

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import json
import os
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

## Importação dos dados

In [ ]:
# Autoriza acesso ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Extrai o dfs da pasta refined
file_path = '/content/drive/MyDrive/dados/csv/refined/data_processed.csv'
df_processed = pd.read_csv(file_path)
df_processed.head()

,cnae,total_acidentes,com_cat_registrada,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,sem_cat_registrada,ano,total_acidentes_seguinte,com_cat_registrada_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte,sem_cat_registrada_seguinte
0,910,1161,1104,900,141,63,57,2014,828,828,708,110,10,0
1,910,828,828,708,110,10,0,2015,562,525,454,56,15,37
2,910,562,525,454,56,15,37,2016,405,366,307,47,12,39
3,910,405,366,307,47,12,39,2017,517,479,408,45,26,38
4,910,517,479,408,45,26,38,2018,391,369,319,43,7,22


## 1. Divisão dos dados para treinamento

In [ ]:
# Garante que o df está ordenado por ano (importante para o TimeSeriesSplit)
df_processed = df_processed.sort_values(by='ano')
df_processed.head(2)

,cnae,total_acidentes,com_cat_registrada,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,sem_cat_registrada,ano,total_acidentes_seguinte,com_cat_registrada_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte,sem_cat_registrada_seguinte
0,910,1161,1104,900,141,63,57,2014,828,828,708,110,10,0
9,1921,1028,992,729,236,27,36,2014,973,967,713,241,13,6


In [ ]:
df_processed.tail(2)

,cnae,total_acidentes,com_cat_registrada,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,sem_cat_registrada,ano,total_acidentes_seguinte,com_cat_registrada_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte,sem_cat_registrada_seguinte
80,4940,40,40,32,7,1,0,2022,43,41,19,22,0,2
89,7112,3948,3754,2976,676,102,194,2022,5193,4895,3853,904,138,298


In [ ]:
# Define as colunas atributos (total de acidentes, casos típicos, de trajeto e doenças ocupacionais)
colunas_alvo = [
    'total_acidentes',
    # 'com_cat_registrada',
    'acidentes_tipicos',
    'acidentes_trajeto',
    'doenca_trabalho'
    # 'sem_cat_registrada'
]

In [ ]:
# Seleção dos Atributos (X)
X_data = df_processed[colunas_alvo]
X_data.head()

,total_acidentes,acidentes_tipicos,acidentes_trajeto,doenca_trabalho
0,1161,900,141,63
9,1028,729,236,27
27,374,233,60,3
18,646,487,69,24
54,2943,2148,263,22


In [ ]:
# Seleção dos Targets (y)
y_data = df_processed[[col + '_seguinte' for col in colunas_alvo]]
y_data.head()

,total_acidentes_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte
0,828,708,110,10
9,973,713,241,13
27,290,231,57,2
18,365,316,37,12
54,1494,1298,185,11


In [ ]:
print("Dimensões de X (atributos):", X_data.shape)
print("Dimensões de y (alvos múltiplos):", y_data.shape)

Dimensões de X (atributos): (90, 4)
Dimensões de y (alvos múltiplos): (90, 4)


## 2. Inicializa o TimeSeriesSplit

In [ ]:
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)

print(f"Inicia a Validação Cruzada Sequencial com {n_splits} folds.")

Inicia a Validação Cruzada Sequencial com 5 folds.


In [ ]:
# Listas para armazenar as métricas de cada fold
scores_r2 = []
scores_mae = []

fold = 1
for train_index, test_index in tscv.split(X_data):
    # Separa os dados de treino e teste para este fold específico
    X_train, X_test = X_data.iloc[train_index], X_data.iloc[test_index]
    y_train, y_test = X_data.iloc[train_index], X_data.iloc[test_index]

    print(f"\n--- Fold {fold}/{n_splits} ---")
    print(f"  Tamanho do Treino: {len(X_train)} amostras")
    print(f"  Tamanho do Teste: {len(X_test)} amostras")

    # Treina o modelo (é importante criar um novo modelo a cada fold)
    modelo_cv = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo_cv.fit(X_train, y_train)

    # Faz a predição e avalia
    y_pred = modelo_cv.predict(X_test)

    # Calcula as métricas para este fold
    # Para o R², calculamos a média entre todos os alvos
    r2_fold = r2_score(y_test, y_pred)
    # Para o MAE, também calculamos a média
    mae_fold = mean_absolute_error(y_test, y_pred)

    scores_r2.append(r2_fold)
    scores_mae.append(mae_fold)

    print(f"  R² (média dos alvos): {r2_fold:.2f}")
    print(f"  MAE (média dos alvos): {mae_fold:.2f}")

    fold += 1


--- Fold 1/5 ---
  Tamanho do Treino: 15 amostras
  Tamanho do Teste: 15 amostras
  R² (média dos alvos): 0.77
  MAE (média dos alvos): 49.99

--- Fold 2/5 ---
  Tamanho do Treino: 30 amostras
  Tamanho do Teste: 15 amostras
  R² (média dos alvos): 0.95
  MAE (média dos alvos): 20.07

--- Fold 3/5 ---
  Tamanho do Treino: 45 amostras
  Tamanho do Teste: 15 amostras
  R² (média dos alvos): 0.88
  MAE (média dos alvos): 20.34

--- Fold 4/5 ---
  Tamanho do Treino: 60 amostras
  Tamanho do Teste: 15 amostras
  R² (média dos alvos): 0.83
  MAE (média dos alvos): 23.41

--- Fold 5/5 ---
  Tamanho do Treino: 75 amostras
  Tamanho do Teste: 15 amostras
  R² (média dos alvos): 0.75
  MAE (média dos alvos): 75.51


In [ ]:
# Exibe os resultados finais da Validação Cruzada
print("\n--- Resultados Finais da Validação Cruzada ---")
print(f"R² Médio: {np.mean(scores_r2):.2f} (Desvio Padrão: {np.std(scores_r2):.2f})")
print(f"MAE Médio: {np.mean(scores_mae):.2f} (Desvio Padrão: {np.std(scores_mae):.2f})")


--- Resultados Finais da Validação Cruzada ---
R² Médio: 0.84 (Desvio Padrão: 0.07)
MAE Médio: 37.86 (Desvio Padrão: 21.89)


*O modelo de regressão Random Forest demonstrou um desempenho preditivo forte e estável, conforme validado por um processo de validação cruzada para séries temporais. Com um R² médio de 0.84 e baixo desvio padrão de 0.07, o modelo consistentemente explica a maior parte da variação no número de acidentes. O erro médio absoluto (MAE) de 37.86 indica a magnitude do erro esperado, cuja variabilidade (desvio padrão de 21.89) sugere que a precisão das previsões pode flutuar dependendo de fatores anuais específicos."*

## 3. Treinamento do modelo => **Etapa modificada devido a inclusão da validação cruzada**
*Desconsiderar os códigos comentados*

In [ ]:
# df_processed.ano.unique()

array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [ ]:
# Divisão temporal dos dados
# train_indices = df_processed[df_processed['ano'] <= 2021].index
# test_indices = df_processed[df_processed['ano'] == 2022].index

# X_train, X_test = X_data.loc[train_indices], X_data.loc[test_indices]
# y_train, y_test = y_data.loc[train_indices], y_data.loc[test_indices]

In [ ]:
# # Inicializa e treina o modelo
# modelo_multi = RandomForestRegressor(n_estimators=100, random_state=42)
# modelo_multi.fit(X_train, y_train)

# print("Modelo de Regressão Multi-alvo treinado com sucesso!")

Modelo de Regressão Multi-alvo treinado com sucesso!


In [ ]:
# Inicializa o modelo
modelo_final = RandomForestRegressor(n_estimators=100, random_state=42)

# 2. Treina o modelo com TODOS os dados disponíveis
modelo_final.fit(X_data, y_data)

print("\nModelo final treinado com sucesso!")


Modelo final treinado com sucesso!


## Avaliação do Modelo => **Etapa descartada devido a inclusão da validação cruzada que serve para avaliar o modelo de forma mais robusta**

In [ ]:
# X_test.columns

Index(['total_acidentes', 'acidentes_tipicos', 'acidentes_trajeto',
       'doenca_trabalho'],
      dtype='object')

In [ ]:
# X_test.shape

(10, 4)

*As previsões serão um array com 6 colunas*

In [ ]:
# Faz as previsões
# y_pred = modelo_multi.predict(X_test)
# y_pred.shape

(10, 4)

In [ ]:
# Cria um df com as previsões
# df_pred = pd.DataFrame(y_pred, columns=y_test.columns, index=y_test.index)
# df_pred.shape

(10, 4)

In [ ]:
# Itera sobre cada coluna de alvo para calcular as métricas individualmente
# metricas = {}
# for i, col_name in enumerate(y_test.columns):
#     mae = mean_absolute_error(y_test[col_name], df_pred[col_name])
#     r2 = r2_score(y_test[col_name], df_pred[col_name])

#     metricas[col_name.replace('_seguinte', '').upper()] = {
#         'Erro Medio Absoluto (MAE)': mae,
#         'Coeficiente R2': r2
#     }

#     print(f"--- {col_name.replace('_seguinte', '').upper()} ---")
#     print(f"  Erro Médio Absoluto (MAE): {mae:.2f}")
#     print(f"  Coeficiente R²: {r2:.2f}\n")

--- TOTAL_ACIDENTES ---
  Erro Médio Absoluto (MAE): 428.64
  Coeficiente R²: 0.83

--- ACIDENTES_TIPICOS ---
  Erro Médio Absoluto (MAE): 309.16
  Coeficiente R²: 0.84

--- ACIDENTES_TRAJETO ---
  Erro Médio Absoluto (MAE): 75.16
  Coeficiente R²: 0.87

--- DOENCA_TRABALHO ---
  Erro Médio Absoluto (MAE): 14.72
  Coeficiente R²: 0.61



In [ ]:
# Busca as colunas 'cnae' e 'ano' do DataFrame original usando o índice do conjunto de teste
# identificadores = df_processed.loc[y_test.index, ['cnae', 'ano']]
# identificadores.head()

,cnae,ano
8,910,2022
17,1921,2022
26,2513,2022
35,3321,2022
44,3520,2022


In [ ]:
# df_pred.head(2)

,total_acidentes_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte
8,321.45,241.32,29.07,24.96
17,142.80,76.70,31.84,5.78


In [ ]:
# Converte colunas para int
# df_pred = df_pred.astype(int)
# df_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 8 to 89
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   total_acidentes_seguinte    10 non-null     int64
 1   acidentes_tipicos_seguinte  10 non-null     int64
 2   acidentes_trajeto_seguinte  10 non-null     int64
 3   doenca_trabalho_seguinte    10 non-null     int64
dtypes: int64(4)
memory usage: 400.0 bytes


In [ ]:
# Renomea as colunas de previsão
# y_pred_renomeado = df_pred.rename(columns=lambda c: c.replace('_seguinte', '_previsto'))
# y_pred_renomeado.columns

Index(['total_acidentes_previsto', 'acidentes_tipicos_previsto',
       'acidentes_trajeto_previsto', 'doenca_trabalho_previsto'],
      dtype='object')

In [ ]:
# Junta tudo em um único df de resultados
# df_resultados_completos = pd.concat([identificadores, y_test, y_pred_renomeado], axis=1)
# df_resultados_completos.head(2)

,cnae,ano,total_acidentes_seguinte,acidentes_tipicos_seguinte,acidentes_trajeto_seguinte,doenca_trabalho_seguinte,total_acidentes_previsto,acidentes_tipicos_previsto,acidentes_trajeto_previsto,doenca_trabalho_previsto
8,910,2022,490,357,62,31,321,241,29,24
17,1921,2022,265,111,72,22,142,76,31,5


In [ ]:
# df_resultados_completos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 8 to 89
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   cnae                        10 non-null     int64
 1   ano                         10 non-null     int64
 2   total_acidentes_seguinte    10 non-null     int64
 3   acidentes_tipicos_seguinte  10 non-null     int64
 4   acidentes_trajeto_seguinte  10 non-null     int64
 5   doenca_trabalho_seguinte    10 non-null     int64
 6   total_acidentes_previsto    10 non-null     int64
 7   acidentes_tipicos_previsto  10 non-null     int64
 8   acidentes_trajeto_previsto  10 non-null     int64
 9   doenca_trabalho_previsto    10 non-null     int64
dtypes: int64(10)
memory usage: 880.0 bytes


In [ ]:
# Renomea as colunas
# df_resultados_completos = df_resultados_completos.rename(columns=lambda c: c.replace('_seguinte', ''))
# df_resultados_completos.columns

Index(['cnae', 'ano', 'total_acidentes', 'acidentes_tipicos',
       'acidentes_trajeto', 'doenca_trabalho', 'total_acidentes_previsto',
       'acidentes_tipicos_previsto', 'acidentes_trajeto_previsto',
       'doenca_trabalho_previsto'],
      dtype='object')

In [ ]:
# Adequa o ano
# df_resultados_completos['ano'] = df_resultados_completos['ano'].replace(2022,2023)
# df_resultados_completos.head()

,cnae,ano,total_acidentes,acidentes_tipicos,acidentes_trajeto,doenca_trabalho,total_acidentes_previsto,acidentes_tipicos_previsto,acidentes_trajeto_previsto,doenca_trabalho_previsto
8,910,2023,490,357,62,31,321,241,29,24
17,1921,2023,265,111,72,22,142,76,31,5
26,2513,2023,459,378,51,5,521,397,75,8
35,3321,2023,897,698,110,5,679,503,82,11
44,3520,2023,56,27,23,0,39,22,9,0


In [ ]:
# df_resultados_completos.shape

(10, 10)

In [ ]:
# df_resultados_completos = df_resultados_completos.iloc[:,[0,1,2,8,3,9,4,10,5,11,6,12,7,13]]
df_resultados_completos = df_resultados_completos.iloc[:,[0,1,2,6,3,7,4,8,5,9]]
df_resultados_completos.head()

,cnae,ano,total_acidentes,total_acidentes_previsto,acidentes_tipicos,acidentes_tipicos_previsto,acidentes_trajeto,acidentes_trajeto_previsto,doenca_trabalho,doenca_trabalho_previsto
8,910,2023,490,321,357,241,62,29,31,24
17,1921,2023,265,142,111,76,72,31,22,5
26,2513,2023,459,521,378,397,51,75,5,8
35,3321,2023,897,679,698,503,110,82,5,11
44,3520,2023,56,39,27,22,23,9,0,0


## 4. Amazenamento no drive => **Etapa modificada devido a inclusão da validação cruzada**
*Desconsiderar os códigos comentados*

In [ ]:
# Define o caminho e o nome da pasta no Google Drive
caminho_drive = '/content/drive/MyDrive/dados/output/'

In [ ]:
# Salva o Modelo Treinado
caminho_modelo = os.path.join(caminho_drive, 'modelo_random_forest_multi.joblib')
joblib.dump(modelo_final, caminho_modelo)

print(f"Modelo salvo com sucesso em: {caminho_modelo}")

Modelo salvo com sucesso em: /content/drive/MyDrive/dados/output/modelo_random_forest_multi.joblib


In [ ]:
# Salva o dicionário de métricas em JSON
# caminho_metricas = os.path.join(caminho_drive, 'metricas_modelo.json')
# with open(caminho_metricas, 'w') as f:
#     json.dump(metricas, f, indent=4) # indent=4 para deixar o arquivo bem formatado

# print(f"Métricas salvas com sucesso em: {caminho_metricas}")

Métricas salvas com sucesso em: /content/drive/MyDrive/dados/output/metricas_modelo.json


In [ ]:
# Salva as previsões de teste
# file_path = f'{caminho_drive}results_test.csv'
# df_resultados_completos.to_csv(file_path, index=False)

# print(f"Arquivo foi salvo com sucesso em: {file_path}")

Arquivo foi salvo com sucesso em: /content/drive/MyDrive/dados/output/results_test.csv


In [ ]:
# Crie um novo dicionário com as métricas da Validação Cruzada
metricas_cv = {
    "R2_CrossValidation": {
        "media": np.mean(scores_r2),
        "desvio_padrao": np.std(scores_r2)
    },
    "MAE_CrossValidation": {
        "media": np.mean(scores_mae),
        "desvio_padrao": np.std(scores_mae)
    }
}

# Salva o dicionário em um arquivo JSON
caminho_metricas_cv = os.path.join(caminho_drive, 'metricas_validacao_cruzada.json')

with open(caminho_metricas_cv, 'w') as f:
    json.dump(metricas_cv, f, indent=4) # indent=4 para deixar o arquivo bem formatado e legível

print(f"Métricas da validação cruzada salvas com sucesso em: {caminho_metricas_cv}")

# Para visualização, vamos imprimir o conteúdo do dicionário que foi salvo
print("\nConteúdo salvo no arquivo JSON:")
print(json.dumps(metricas_cv, indent=4))

Métricas da validação cruzada salvas com sucesso em: /content/drive/MyDrive/dados/output/metricas_validacao_cruzada.json

Conteúdo salvo no arquivo JSON:
{
    "R2_CrossValidation": {
        "media": 0.8381961217300038,
        "desvio_padrao": 0.07448275976741814
    },
    "MAE_CrossValidation": {
        "media": 37.86466666666668,
        "desvio_padrao": 21.893368793109737
    }
}
